プロンプトテンプレートは、言語モデル用のプロンプトを生成するための事前定義されたレシピです。

> Prompt templates are predefined recipes for generating prompts for language models.

テンプレートには、指示、数ショットの例、特定のコンテキスト、および特定のタスクに適した質問が含まれることがあります。

> A template may include instructions, few-shot examples, and specific context and
> questions appropriate for a given task.

LangChainは、プロンプトテンプレートを作成し、作業するためのツールを提供します。

> LangChain provides tooling to create and work with prompt templates.

LangChainは、異なる言語モデル間で既存のテンプレートを簡単に再利用できるように、モデル非依存のテンプレートを作成することに努めています。

> LangChain strives to create model agnostic templates to make it easy to reuse
> existing templates across different language models.

通常、言語モデルはプロンプトが文字列であるか、あるいはチャットメッセージのリストであることを期待しています。

> Typically, language models expect the prompt to either be a string or else a list of chat messages.

## `PromptTemplate` | `PromptTemplate`

`PromptTemplate`を使用して、文字列プロンプトのテンプレートを作成します。

> Use `PromptTemplate` to create a template for a string prompt.

デフォルトでは、`PromptTemplate`は[Pythonのstr.format](https://docs.python.org/3/library/stdtypes.html#str.format)の構文をテンプレートに使用します。

> By default, `PromptTemplate` uses [Python's str.format](https://docs.python.org/3/library/stdtypes.html#str.format)
> syntax for templating.



In [17]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

このテンプレートは、変数がない場合を含め、任意の数の変数をサポートしています。

> The template supports any number of variables, including no variables:



In [18]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

さらなる検証のために、`input_variables`を明示的に指定してください。これらの変数は、インスタンス化の際にテンプレート文字列内に存在する変数と比較され、**不一致がある場合には例外が投げられます**。例えば：

> For additional validation, specify `input_variables` explicitly. These variables
> will be compared against the variables present in the template string during instantiation, **raising an exception if
> there is a mismatch**. For example:



In [19]:
from langchain.prompts import PromptTemplate

invalid_prompt = PromptTemplate(
    input_variables=["adjective"],
    template="Tell me a {adjective} joke about {content}.",
)

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. 'content' (type=value_error)

好きな方法でプロンプトをフォーマットするカスタムプロンプトテンプレートを作成できます。詳細は、[カスタムプロンプトテンプレート](./custom_prompt_template.html)を参照してください。

> You can create custom prompt templates that format the prompt in any way you want.
> For more information, see [Custom Prompt Templates](./custom_prompt_template.html).

## `ChatPromptTemplate` | `ChatPromptTemplate`

[チャットモデル](../models/chat)へのプロンプトは、チャットメッセージのリストです。

> The prompt to [chat models](../models/chat) is a list of chat messages.

各チャットメッセージにはコンテンツが関連付けられており、`role` という追加のパラメーターがあります。例えば、OpenAI の [Chat Completions API](https://platform.openai.com/docs/guides/chat/introduction) では、チャットメッセージは AI アシスタント、人間、またはシステムの役割に割り当てられます。

> Each chat message is associated with content, and an additional parameter called `role`.
> For example, in the OpenAI [Chat Completions API](https://platform.openai.com/docs/guides/chat/introduction), a chat message can be associated with an AI assistant, a human or a system role.

このようなチャットプロンプトのテンプレートを作成してください：

> Create a chat prompt template like this:



In [22]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

`ChatPromptTemplate.from_messages`は、さまざまなメッセージ表現を受け入れます。

> `ChatPromptTemplate.from_messages` accepts a variety of message representations.

たとえば、上記で使用された（type, content）の2タプル表現に加えて、`MessagePromptTemplate` や `BaseMessage` のインスタンスを渡すことができます。

> For example, in addition to using the 2-tuple representation of (type, content) used
> above, you could pass in an instance of `MessagePromptTemplate` or `BaseMessage`.



In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

llm = ChatOpenAI()
llm(chat_template.format_messages(text="i dont like eating tasty things."))

AIMessage(content='I absolutely love indulging in delicious treats!')

これにより、チャットプロンプトを構築する方法において、大きな柔軟性が得られます。

> This provides you with a lot of flexibility in how you construct your chat prompts.



## LCEL | LCEL

`PromptTemplate`と`ChatPromptTemplate`は、[LangChain Expression Language (LCEL)](/docs/expression_language/)の基本的な構成要素である[Runnable interface](/docs/expression_language/interface)を実装しています。これは、`invoke`、`ainvoke`、`stream`、`astream`、`batch`、`abatch`、`astream_log`の呼び出しをサポートしていることを意味します。

> `PromptTemplate` and `ChatPromptTemplate` implement the [Runnable interface](/docs/expression_language/interface), the basic building block of the [LangChain Expression Language (LCEL)](/docs/expression_language/). This means they support `invoke`, `ainvoke`, `stream`, `astream`, `batch`, `abatch`, `astream_log` calls.

`PromptTemplate`はプロンプト変数の辞書を受け取り、`StringPromptValue`を返します。`ChatPromptTemplate`は辞書を受け取り、`ChatPromptValue`を返します。

> `PromptTemplate` accepts a dictionary (of the prompt variables) and returns a `StringPromptValue`. A `ChatPromptTemplate` accepts a dictionary and returns a `ChatPromptValue`.



In [24]:
prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val

StringPromptValue(text='Tell me a joke')

In [25]:
prompt_val.to_string()

'Tell me a joke'

In [26]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a joke')]

In [30]:
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})

In [31]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

In [32]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."